In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
sns.set(rc={'figure.figsize':(11,9)})
%matplotlib inline

# 0 for GPU, -1 for CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from LSTM_forecast import load_csv_data, load_mat_data, preprocess, split_signals, data_labeling
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, precision_score, 
                             average_precision_score, roc_auc_score, roc_curve, RocCurveDisplay)
%load_ext tensorboard
%load_ext snakeviz

## Data Visualization

In [ ]:
df = load_csv_data('data/trainSet', concat=True)
df_list = load_csv_data('data/trainSet', concat=False)
df.head()
#df_list[0].iloc[:,:1].head(20)
#df.dtypes
#print(df_list[0].columns.tolist())

In [ ]:
#df = load_mat_data('data/trainSet')
df = load_mat_data('data/testSet')
df.head()
#df.plot()

In [ ]:
# change data type to numeric is needed to plot data frame (only trainSet)
for i in range(len(df_list)):
    df_list[i]['M [Nm]'] = pd.to_numeric(df_list[i]['M1 in Nm '])
    df_list[i]['Temp in [°C]'] = pd.to_numeric(df_list[i]['Temp A  '])
    df_list[i]['Temp out [°C]'] = pd.to_numeric(df_list[i]['Temp B  '])
    df_list[i]['Current [A]'] = pd.to_numeric(df_list[i]['Strom in A'])
    df_list[i]['n [1/min]'] = pd.to_numeric(df_list[i]['n in 1/min'])
    df_list[i]['Cycles'] = pd.to_numeric(df_list[i]['Zyklen'])

### Sensor Signals

In [ ]:
# trainData (real measurements)
cols_plot = ['M [Nm]', 'Temp in [°C]', 'Temp out [°C]', 'Current [A]', 'n [1/min]']
axes = df_list[6][cols_plot].plot(linewidth=1, figsize=(15,10), subplots=True)
for i, ax in enumerate(axes):
    ax.set_ylabel(cols_plot[i])
    ax.set_xlabel('Sample No.')
#plt.savefig('realMeasurement.pdf', bbox_inches='tight')

In [ ]:
# testData (physical Model)
df['Current [A]'] = df['current_A']
df['Position [m]'] = df['position_m']
df['n [1/sec]'] = df['rotspeed_persec']
df['Acceleration [rad/(sec^2)]'] = df['acceleration_radpersec2']
#cols_plot = ['current_A', 'position_m', 'rotspeed_persec', 'acceleration_radpersec2']
cols_plot = ['Current [A]', 'Position [m]', 'n [1/sec]', 'Acceleration [rad/(sec^2)]']
axes = df[cols_plot].plot(linewidth=1, figsize=(15,10), subplots=True)
for i, ax in enumerate(axes):
    ax.set_ylabel(cols_plot[i])
    ax.set_xlabel('Sample No.')
    #ax.axvspan(8450, 9750, color='red', alpha=0.2)
    #ax.axvspan(5460, 6300, color='red', alpha=0.2)
#plt.savefig('CollisionData.pdf', bbox_inches='tight')

### Heatmap (Correlation Matrix)

In [ ]:
# to use only the most relevant sensor signals 
data = pd.concat((df_list), axis=0, ignore_index=True)
data = data.drop(columns=['Cycles'])
#data = df_list[1]
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(15,10))
heatmap = sns.heatmap(data[top_corr_features].corr(), annot=True, cmap='RdYlGn')
#plt.savefig('correlation.pdf', bbox_inches='tight')

## LSTM Forecast Model

In [ ]:
model = load_model('models/lstm_current_rotspeed_pos_accelerat')

### Model Architectur 

In [ ]:
print(model.summary())

In [ ]:
plot_model(
    model,
    to_file='modelStruct.pdf',
    show_shapes=True,
    show_dtype=True,
    show_layer_names=False,
    rankdir='LR', #TB, LR
    expand_nested=False,
    dpi=96
)

### Training History

In [ ]:
with open('models/history_lstm_current_rotspeed_pos_accelerat.pkl', 'rb') as f:
        history = pickle.load(f)
print(history.keys())

In [ ]:
from scipy.signal import savgol_filter
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5), sharex=False, sharey=False)

window = 3
order = 1
loss_smoothed = savgol_filter(history['loss'], window, order)
val_loss_smoothed = savgol_filter(history['val_loss'], window, order)

ax1.plot(loss_smoothed, color=sns.color_palette()[1])
ax1.plot(val_loss_smoothed, color=sns.color_palette()[2])
ax1.legend(['training', 'validation'])
ax1.plot(history['loss'], color=sns.color_palette()[1], alpha=0.3)
ax1.plot(history['val_loss'], color=sns.color_palette()[2], alpha=0.3)
#ax1.set_title('model loss')
ax1.set_ylabel('Loss (MAE)')
ax1.set_xlabel('Epoch')

ax2.plot(history['lr'])
#ax2.set_title('model learning rate')
ax2.set_ylabel('Learning Rate')
ax2.set_xlabel('Epoch')
#plt.savefig('training.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#%%cmd
#rm -rf ./logs/
!rm -rf ./logs/

In [ ]:
%tensorboard --logdir logs/fit/ --port=8008

### Prediction (Forecast and Detection)

In [ ]:
SENSITIV = 1.25 # sensitivity parameter to calibrate the prediction threshold
SEQ_LEN = 100 # every input instance is a sequence of length n

#### Forecasting Traindata

In [ ]:
#inputs, outputs, _ = preprocess('data/trainSet', record=0, file='csv')
inputs, outputs, num_features , original_in, original_out, scaler = preprocess('data/trainSet', record=None, file='mat')

preds = model.predict(inputs, verbose=1, batch_size=50)
mse_perSignal = ((outputs - preds) ** 2)/num_features
mae = pd.DataFrame(outputs - preds).abs().sum(axis=1).divide(num_features)
quantile = mae.quantile(0.99)

with open('models/train_Settings.pkl', 'wb') as f:
    pickle.dump([num_features, scaler, quantile], f)

#### Forecasting Testdata 

In [ ]:
#%%snakeviz
# use standardscaler and quantile (threshold) from trainset 
with open('models/train_Settings.pkl', 'rb') as f:
    num_features, scaler, quantile = pickle.load(f)
data = load_mat_data('data/testSet')
data = data.drop(columns=['position_rad', 'temperature_DegCel'])
inputs, outputs = split_signals(scaler.transform(data), SEQ_LEN)

preds = model.predict(inputs, verbose=1, batch_size=1)
mse_perSignal = ((outputs - preds) ** 2)/num_features
mae = pd.DataFrame(outputs - preds).abs().sum(axis=1).divide(num_features)
#quantile = 0.15109934516144174

In [ ]:
print(inputs.shape)
print(preds.shape)
print(outputs.shape)
print(mse_perSignal.shape)
print(mae.shape)
print(quantile)

In [ ]:
fig, axs = plt.subplots(4, figsize=(15,10), sharex=True, sharey=False)
#plot_names = ['Temp in', 'Strom in A', 'n in 1/min', 'MSE'] # csv
plot_names = ['Acceleration [rad/(sec^2)]', 'Current [A]', 'Position [m]', 'n [1/sec]'] # mat

for k, ax in enumerate(axs):
    ax.set_ylabel(plot_names[k])
#    if k == 4:
#        ax.plot(mse_perSignal[:,1], c=sns.color_palette()[3], label='Current')
#        continue
    ax.plot(pd.Series(np.ravel(outputs[:len(outputs),k])), c=sns.color_palette()[0], label='Actual')
    ax.plot(pd.Series(np.ravel(preds[:len(preds),k])), c=sns.color_palette()[1], label='Forecast')
    ax.legend(loc='lower left')
    #ax.axvspan(5460, 6300, color='red', alpha=0.2)
    ax.axvspan(8350, 9650, color='red', alpha=0.2)
ax.set_xlabel('Sample No.')
#plt.savefig('forecastTestDC.pdf', bbox_inches='tight')
plt.show()

#### Detection

In [ ]:
ax = mae.plot(figsize=(15,5))
#ax.set_title('Summed Error for all Sensor Signals')
ax.set_xlabel('Sample No.')
ax.set_ylabel('MAE')
ax.hlines(y=quantile, xmin=0, xmax=len(preds), colors='g', linestyles='--', lw=2)
ax.hlines(y=SENSITIV*quantile, xmin=0, xmax=len(preds), colors='r', linestyles='--', lw=2)
ax.axvspan(5460, 6300, color='red', alpha=0.2)
#ax.axvspan(8350, 9650, color='red', alpha=0.2)
#plt.savefig('detectionTestDC.pdf', bbox_inches='tight')
plt.show()

#### Metrics

In [ ]:
#df_evaluate = data_labeling(path='data/testSet', sum_error=mae, quantile=quantile, startIdx=5460)
df_evaluate = data_labeling(path='data/testSet', sum_error=mae, quantile=quantile, startIdx=8350)
gt = df_evaluate['ground truth']
preds = df_evaluate['prediction']
scores = df_evaluate['score']
#df_evaluate.head(-1)

In [ ]:
print('Precision Score:          %.4f' % precision_score(gt, preds))
print('Recall Score:             %.4f' % recall_score(gt, preds))
print('F1 Score:                 %.4f' % f1_score(gt, preds))
print('Average Precision Score:  %.4f' % average_precision_score(gt, scores))
print('ROC AUC Score:            %.4f' % roc_auc_score(gt, scores))

In [ ]:
sns.reset_orig()
tn, fp, fn, tp = confusion_matrix(gt, preds).ravel()
tpr = tp / (tp + fn)
fnr = fn / (fn + tp)
fpr = fp / (fp + tn)
tnr = tn / (tn + fp)
print('True Positive Rate:       %.4f' % tpr)
print('False Negative Rate:      %.4f' % fnr)
print('False Positive Rate:      %.4f' % fpr)
print('True Negative Rate:       %.4f' % tnr)
cm = confusion_matrix(gt, preds)
ConfusionMatrixDisplay(cm).plot()
#plt.savefig('confMatrixTestCollision.pdf', bbox_inches='tight')

In [ ]:
sns.set(rc={'figure.figsize':(11,9)})
fpr, tpr, _ = roc_curve(gt, scores)
RocCurveDisplay(fpr=fpr, tpr=tpr,).plot()
#plt.savefig('ROCTestDC.pdf', bbox_inches='tight')